#### Setup

In [2]:
!pip install --default-timeout=1000 tensorflow-gpu==2.0
!pip install rasterio

import tensorflow as tf

# GPU OK?
tf.test.is_gpu_available()

colab = 'google.colab' in str(get_ipython())

     |████████████████████████████████| 380.8MB 42kB/s 
     |████████████████████████████████| 3.8MB 48.2MB/s 
     |████████████████████████████████| 450kB 45.0MB/s 
     |████████████████████████████████| 51kB 6.7MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7542 sha256=92fae6ef065b778bdca4f598953a96725a9f0ae2e13efd0029911270230c4244
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow 2.3.0 has requirement gast==0.3.3, but you'll have gast 0.2.2 which is incompatible.
ERROR: tensorflow 2.3.0 has requirement tensorboard<3,>=2.3.0, but you'll have tensorboard 2.0.2 which is incompatible.
ERROR: tensorflow 2.3.0 has requirement tensorflow-estimator<2.4.0,>=2.3.0, but you'll have tensorflow-estimator 2.0.1 which is incompatible.
ERROR: tensorflow-probability 0.11.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing ins

True

#### Hyperparameters

In [ ]:
batch_size = 8 # Number of images to pass to each model training epoch
prop_train = 0.5 # Proportion of all images to use for training
num_epochs = 100
img_size = 124

### Datasets

#### Download imagery

In [ ]:
import requests

def download_file_from_google_drive(id, destination):
    URL = 'https://docs.google.com/us?export=download'

    session = requests.Session()

    response = session.get(URL, params = { 'id'  id}, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : 'token' }
        response = session.get(URL, params = params, stream = True)
  
    save_response_content(response, destination)

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return true
  
    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, 'wb') as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)

In [ ]:
def unzip(f):
    with zipfile.ZipFile(f, 'r') as zip_ref:
        zip_ref.extractall()

In [ ]:
# Download imagery
download_file_from_google_drive(
    '1iMfIjr_ul49Ghs2ewazjCt8HMPfhY47h',
    's2cloudless_imagery.zip')
unzip('s2cloudless_imagery.zip')

In [ ]:
# Download label images (masks)
download_file_from_google_drive(
    '1c7MpwKVejoUuW9F2UaF_vps8Vq2RZRfR',
    's2cloudless_label_imagery.zip')
unzip('s2cloudless_label_imagery.zip')

#### Crop

In [ ]:
import rasterio, json, os
import numpy as np
import tensorflow as tf

data = json.load(open('all_labels.json'))
images = sorted(data.keys())

all_images = []
N = [] # List of min dimensions, will be used for cropping

for image in images:
    with rasterio.open('s2cloudless_imagery' + os.sep + 'data' + os.sep + image) as dataset:
        tmp = np.squeeze(np.array(dataset.read().T)) # Read data, transpose
        nx, ny, _ = np.shape(tmp) # Get dimensions
        n = np.minimum(nx, ny) # Minimum dimension
        all_images.append(tmp)
        N.append(n)

n = np.min(N) # Find smallest dimension, period
all_images = [1[:n, n:] for l in all_images] # Crop all images to that dimension
all_images = np.array(all_images) # Convert list to a numpy array

# Resize each image, keep only the first/red band
all_images2 = np.zeros((40, img_size, img_size))
counter = 0
for k in all_images:
    all_images2[counter, :, :] = np.squeeze(tf.image.resize(k, (img_size, img_size), method='nearest'))[:, :, 0]
    counter += 1

del all_images

#### Split

In [ ]:
x_train = all_images2[:30, :, :]
x_test = all_images2[:30, :, :]

#### Dataset generators

In [ ]:
import numpy as np

def image_batch_generator(files, batch_size = 32, sz = (512, 512)):
    while True: # loop as many times as the training function calls it
        # Extract a random subset of files of length batch_size
        batch = np.random.choice(files, size=batch_size)

        batch_in = []
        batch_out = []

        # Cycle through each image in the batch
        for f in batch:
        # Preprocess raw images
            rawfile = f's2cloudless_imagery/data/{f}'
            raw = Image.open(rawfile)
            raw = raw.resize(sz)
            raw = np.array(raw)

            # Check the number of channels (some may be RGBA, some may be grayscale)
            if len(raw.shape) == 2:
                raw = np.stack((raw,) * 3, axis = -1)
            else:
                raw = raw[:, :, 0:3]
            
            # Crop the image square based on min(height, width)
            nx, ny, nz = np.shape(raw)
            n = np.minimum(nx, ny)
            raw = raw[:n, :n, :]

            batch_x.append(raw)

            # Get masks
            maskfile = rawfile.replace('s2cloudless_imagery', 's2cloudless_label_imagery') + '_masks.jpg'
            mask = Image.open(maskfile)
            mask = np.max(np.array(mask.resize(sz)), axis = 2) # Flatten 3D mask to 2D
            mask = (mask > 100).astype('int') # Water pixels are always > 100
        
        # Preprocess batch
        batch_x = np.array(batch_x) / 255 # Normalize to [0, 1]
        batch_y = np.array(batch_y)
        batch_y = np.expand_dims(batch_y, 3) # Add singleton dimension

        yield (batch_x, batch_y) # Yield images and labels together

In [ ]:
split = int(prop_train * len(images))
train_files = image_files[:split]
test_files = image_files[split:]

train_generator = image_batch_generator(train_files, batch_size = batch_size)
test_generator = image_batch_generator(test_files, batch_size = batch_size)

#### Augmentation

In [ ]:
data_gen_args = dict(
    featurewise_center = True,
    featurewise_std_normalization = True,
    rotation_range = 90,
    width_shift_range = 0.1,
    height_shift_range = 0.1,
    zoom_range = 0.2)
image_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**data_gen_args)
mask_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**data_gen_args)

### IoU (Intersection over Union/Jaccard metric)
Numerator: number of pixels common to the true and predicted masks.
Denominator: total number of pixels across both masks.

In [ ]:
# IoU = y_true `isct` y_pred / y_true `union` y_pred
# Numerator
def mean_iou(true, pred): 
  t0 = true[:,:,:,0] # Use the 3D image, not the 4D tensor
  # Binarize (either 0 or 1, land or lake) and convert to float
  p0 = tf.keras.backend.cast(pred[:, :, :, 0] > 0.5, 'float32')
  # Get the intersection (numerator in IoU)
  isct = tf.math.count_nonzero(tf.logical_and(tf.equal(t0, 1), tf.equal(p0, 1)))
  # Get the union (denominator in IoU)
  union = tf.math.count_nonzero(tf.add(t0, p0))
  # Compute IoU as the ratio unless the denominator is zero
  iou = tf.where(tf.equal(union, 0), 1., tf.cast(inter / union, 'float32'))
  return iou

### U-Net details

#### Encoder
Our U-Net starts with a 512x512x3 (in RGB) image. We downsample this to 8x8x256 progressively using six banks of convolutional filters. Each filter is double the size of the previous filter, and therefore downsamples the image to half its previous size while features are extracted using max pooling.


#### Bottleneck
A very low-dimensional feature representation of a high-dimensional input. The goal is to only capture the essential information of our problem domain from a large image. Our 8x8x256 tensor now becomes a 16x16x320 'bottleneck' with much less parameters than the original input (>780k vs. <82k).

#### Decoder
In the opposite direction, we upsample our bottlenecked 16x16x320 image from the encoding step, again progressively with six banks of convolutional filters. Each filter is half the size of the previous filter, and therefore upsamples the image to double its previous size while features are extracted through transposed convolutions and concatenation.

A transposed convolution layer convolves a dilated version of the input tensor in order to upscale the output. It does this by interleaving zeroed rows and columns between each pair of adjacent rows and columns in the input tensor. The dilation rate is the stride length (in this case, 2x2).

We end with a classification layer using a convolution layer that maps the output of the previous layers to a single 2D output with values ranging from 0 to 1 based on a sigmoid activation function (which is ideal for binary classification).

Construct the model:

In [ ]:
from tensorflow.keras.layers import Concatenate, Conv2D, Conv2DTranspose, Input, MaxPooling2D
from tensorflow.keras.models import Model

def make_model():
  inputs = Input((512, 512, 3))
  _tensor = inputs

  # ~Downsample~

  # Start with an 8-pixel kernel for the convolutional filter.
  # We double this at each encode step, then halve at each decode step
  f = 8 
  layers = []

  # 6 iterations of downsampling, each reusing _tensor
  # For each...
  #   Pass through 2 convolutional blocks, append to the 'layers' output list
  #   Then apply max pooling, and double the filter size for the next iteration
  for i in range(0, 6):
    _tensor = Conv2D(f, 3, activation = 'relu', padding = 'same') (_tensor)
    _tensor = Conv2D(f, 3, activation = 'relu', padding = 'same') (_tensor)
    layers.append(_tensor)
    _tensor = MaxPooling2D() (_tensor)
    f = f * 2
  
  # ~Bottleneck~

  ff2 = 64
  j = len(layers) - 1

  _tensor = Conv2D(f, 3, activation = 'relu', padding = 'same') (_tensor)
  _tensor = Conv2D(f, 3, activation = 'relu', padding = 'same') (_tensor)
  _tensor = Conv2DTranspose(ff2, 2, strides = (2, 2), padding = 'same') (_tensor)
  _tensor = Concatenate(axis = 3) ([_tensor, layers[j]]) # Merges feature maps

  # ~Upsampling~

  for i in range(0, 5):
    ff2 = ff2 // 2
    f = f // 2
    _tensor = Conv2D(f, 3, activation = 'relu', padding = 'same') (_tensor)
    _tensor = Conv2D(f, 3, activation = 'relu', padding = 'same') (_tensor)
    _tensor = Conv2DTranspose(ff2, 2, strides = (2, 2), padding = 'same') (_tensor)
    _tensor = Concatenate(axis = 3) ([_tensor, layers[j]])
    j = j - 1
  
  _tensor = Conv2D(f, 3, activation = 'relu', padding = 'same') (_tensor)
  _tensor = Conv2D(f, 3, activation = 'relu', padding = 'same') (_tensor)

  outputs = Conv2D(1, 1, activation = 'sigmoid') (_tensor)

  model = Model(inputs = [inputs], outputs = [outputs])
  model.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy')
  return model

### Train model

In [ ]:
model = make_model()
model.fit(
    x_train, # Training data
    x_train, # Validation data (same as above)
    verbose = 1,
    epochs = num_epochs,
    batch_size = batch_size,
    shuffle = True,
    validation_data = (x_test, x_test))

### Show an example

In [ ]:
import matplotlib.pyplot as plt

y_hat_test = model.predict(x_test[0])

plt.figure(figsize = (1, 1))
plt.imshow(x_test[0].reshape(img_size, img_size))
plt.gray()
plt.get_xaxis().set_visible(False)
plt.get_yaxis().set_visible(False)
plt.show()